# variables

In [35]:
API_URL = "http://127.0.0.1:8006" #  "https://api.staging.osparc.io" #"staging.osparc.io"
WEB_API_URL = "http://127.0.0.1:9081" # "https://staging.osparc.io"
USERNAME = "anderegg@itis.swiss"
PASSWORD = "z431"
API_KEY = "352bd1ff-2513-51b6-8886-3b0effdde766" # "94bfcaef-bba4-51ab-849b-91596b9683a0"
API_SECRET = "2a9db08f-04cb-5343-a211-cd48fe66a316" #"33809e75-8b24-56b7-81cc-e0bdcfffe072"
SOLVER_NAME = "simcore/services/comp/kember-cardiac-model"
SOLVER_VERSION = "1.0.0"
# SOLVER_NAME = "simcore/services/comp/itis/sleeper"
# SOLVER_VERSION = "2.0.2"

CLUSTER_ID = 0


# initialize connection

In [36]:
import osparc
from tenacity import AsyncRetrying, TryAgain, retry_if_not_exception_type
from tenacity.wait import wait_fixed
import asyncio
import typing
import json
import urllib3
from tqdm.notebook import tqdm_notebook


cfg = osparc.Configuration(
    host=f"{API_URL}",
    username=API_KEY,
    password=API_SECRET,
)
# cfg.debug = True

In [37]:
with osparc.ApiClient(cfg) as api_client:

    profile = osparc.UsersApi(api_client).get_my_profile()
    # print(profile)
    

    solvers_api = osparc.SolversApi(api_client)
    original_api_client_call_api = solvers_api.api_client.call_api

    def patched_api_client_for_starting_jobs(*args, **kwargs):
        # print("called with: ", args, kwargs)
        if args[0] == "/v0/solvers/{solver_key}/releases/{version}/jobs/{job_id}:start" and args[1] == "POST":
            query_params = args[3]
            # print("called to start a job, let's add ", CLUSTER_ID, "to", query_params)
            query_params.append(("cluster_id", CLUSTER_ID))
            # print("changed to: ", args, kwargs)
        return original_api_client_call_api(*args, **kwargs)

    solvers_api.api_client.call_api = patched_api_client_for_starting_jobs

# get solver

In [38]:
solver = typing.cast(osparc.Solver, solvers_api.get_solver_release(SOLVER_NAME, SOLVER_VERSION))

print(solver)

{'description': 'Kember cardiac model',
 'id': 'simcore/services/comp/kember-cardiac-model',
 'maintainer': 'anderegg@itis.swiss',
 'title': 'Kember cardiac model',
 'url': 'http://127.0.0.1:8006/v0/solvers/simcore/services/comp/kember-cardiac-model/releases/1.0.0',
 'version': '1.0.0'}


# create job

In [40]:
import functools
from multiprocessing.pool import AsyncResult


async def create_job(solver: osparc.Solver) -> osparc.Job:
    result = await asyncio.get_running_loop().run_in_executor(None, functools.partial(solvers_api.create_job,
            solver.id,
            solver.version,
            osparc.JobInputs(
                {
                    "dt": 1,
                }
            ), async_req=True
        ))
    assert isinstance(result, AsyncResult) # nosec
    # print(job)
    return typing.cast(osparc.Job, await asyncio.get_running_loop().run_in_executor(None, result.get))

# list jobs

In [41]:
async def list_jobs(solver: osparc.Solver) -> list[osparc.Job]:
    result = await asyncio.get_running_loop().run_in_executor(None, functools.partial(solvers_api.list_jobs, solver.id, solver.version, async_req=True))
    assert isinstance(result, AsyncResult) # nosec
    # print([job.id for job in jobs])
    return typing.cast(list[osparc.Job], await asyncio.get_running_loop().run_in_executor(None, result.get))
jobs =await list_jobs(solver)
assert len(jobs) == 0, f"found {len(jobs)} jobs"

# inspect job

In [42]:
async def inspect_job(solver: osparc.Solver, job: osparc.Job)-> osparc.JobStatus:
    result = await asyncio.get_running_loop().run_in_executor(None, functools.partial(solvers_api.inspect_job, solver.id, solver.version, job.id,async_req=True))
    assert isinstance(result, AsyncResult) # nosec
    # print(status)
    return typing.cast(osparc.JobStatus, await asyncio.get_running_loop().run_in_executor(None, result.get))

## get job result

In [43]:
async def get_job_result(solver: osparc.Solver, job: osparc.Job) -> osparc.JobOutputs:
    result = await asyncio.get_running_loop().run_in_executor(None, functools.partial(solvers_api.get_job_outputs, solver.id, solver.version, job.id, async_req=True))
    assert isinstance(result, AsyncResult) # nosecregistry.staging.osparc.io/simcore/services/comp/itis/sleeper:2.0.2
    return typing.cast(osparc.JobOutputs, await asyncio.get_running_loop().run_in_executor(None, result.get))

# start job

In [44]:
async def start_job(solver: osparc.Solver, job: osparc.Job) -> osparc.JobStatus:
    result=  await asyncio.get_running_loop().run_in_executor(None, functools.partial(solvers_api.start_job, solver.id, solver.version, job.id,async_req=True))
    assert isinstance(result, AsyncResult) # nosec
    return typing.cast(osparc.JobStatus, await asyncio.get_running_loop().run_in_executor(None, result.get))

# delete a job corresponding project

In [45]:
def _login_webserver() -> dict[str, str]:
   # create a PoolManager instance
    http = urllib3.PoolManager()

    # define the URL of the login page
    url = f'{WEB_API_URL}/v0/auth/login'

    # define the data to be sent with the POST request
    data = {'email': USERNAME, 'password': PASSWORD}

    # send the POST request to the login page
    response = http.request('POST', url, body=json.dumps(data), headers={'Content-Type': 'application/json'})

    # print the response status code and content
    if response.status != 200:
        raise RuntimeError("failed to login!")
    
    # get the cookies
    cookies = response.headers.get("Set-Cookie")
    return cookies

In [46]:
def delete_jobs(jobs: list[osparc.Job]) -> None:
    # create a PoolManager instance
    http = urllib3.PoolManager()

    # define the URL of the login page
    url = f'{WEB_API_URL}/v0/auth/login'

    # define the data to be sent with the POST request
    data = {'email': USERNAME, 'password': PASSWORD}

    # send the POST request to the login page
    response = http.request('POST', url, body=json.dumps( data), headers={'Content-Type': 'application/json'})

    # print the response status code and content
    if response.status != 200:
        raise RuntimeError("failed to login!")
    # print("logged in, proceeding with deletion...")

    # get the cookies
    cookies = response.headers.get("Set-Cookie")

    # delete a project
    for job in tqdm_notebook(jobs, desc="Deleting jobs"):
        url = f'{WEB_API_URL}/v0/projects/{job.id}'
        response = http.request("DELETE", url, headers={'Content-Type': 'application/json', "Cookie": cookies})
        # print(f"job {job.id} is {'deleted' if response.status == 204 else 'failed to delete'}")


### list jobs through webserver

In [47]:
def list_job_ids_from_web_server() -> list[str]:
    cookies = _login_webserver()
    # create a PoolManager instance
    http = urllib3.PoolManager()

    # list projects that are hidden
    url = f'{WEB_API_URL}/v0/projects?type=user&show_hidden=true'
    offset = 0
    limit = 30
    response = http.request("GET", f"{url}&limit={limit}&offset={offset}", headers={'Content-Type': 'application/json', "Cookie": cookies})
    if response.status != 200:
        raise RuntimeError(f"Unable to list projects from webserver: {response.data}")
    data = json.loads(response.data)

    total = data["_meta"]["total"]
    total_count = data["_meta"]["count"]
    offset += data["_meta"]["count"]
    list_of_job_ids = [project["uuid"] for project in data["data"] if str(project["name"]).startswith("solvers/simcore")]

    while total_count < total:
        response = http.request("GET", f"{url}&limit={limit}&offset={offset}", headers={'Content-Type': 'application/json', "Cookie": cookies})
        if response.status != 200:
            raise RuntimeError(f"Unable to list projects from webserver: {response.data}")
        data = json.loads(response.data)
        list_of_job_ids += [project["uuid"] for project in data["data"] if str(project["name"]).startswith("solvers/simcore")]
        total_count += data["_meta"]["count"]
        offset+=data["_meta"]["count"]
    
    return list_of_job_ids
    # print(list_of_job_ids)

list_of_job_ids = list_job_ids_from_web_server()
print(len(list_of_job_ids))

1


In [48]:
jobs = [osparc.Job(id=job_id, name="fake", inputs_checksum="", created_at="", runner_name="blah", url="asd", runner_url="asd", outputs_url="asd") for job_id in list_of_job_ids]

# run N sleepers

In [76]:
NUM_JOBS = 1
from tqdm.notebook import tqdm

jobs = []
with tqdm(total=NUM_JOBS, desc="Creating jobs") as pbar:
    for result in asyncio.as_completed([create_job(solver) for _ in range(NUM_JOBS)]):
        job = await result
        jobs.append(job)
        pbar.update()


Creating jobs:   0%|          | 0/1 [00:00<?, ?it/s]

In [77]:


async def run_job(solver: osparc.Solver, job: osparc.Job):
    # with tqdm(total=3) as pbar:
    await start_job(solver, job)
    # pbar.update()

    job_status = await inspect_job(solver, job)
    async for attempt in AsyncRetrying(wait=wait_fixed(1), retry=retry_if_not_exception_type()):
        with attempt:
            new_job_status = await inspect_job(solver, job)
            if job_status.state != new_job_status.state:
                job_status = new_job_status
                # pbar.update()
                print(f"job {job.id} currently {job_status}")
            if job_status.progress != new_job_status.progress:
                job_status = new_job_status
                print(f"job progress is now {job_status.progress}")
            if job_status.state not in ["SUCCESS", "FAILED"]:
                raise TryAgain
            if job_status.state == "FAILED":
                raise RuntimeError("job failed!")
            return job_status

job_statuses = []
with tqdm(total=len(jobs), desc="Running jobs") as pbar:
    for result in asyncio.as_completed([run_job(solver, job) for job in jobs]):
        try:
            job_statuses.append(await result)            
        except osparc.ApiException as exc:
            tqdm.write(f"Error while running {job.id}: {exc}")
        finally:
            pbar.update()

Running jobs:   0%|          | 0/1 [00:00<?, ?it/s]

job 78592c3d-a1c9-4293-840f-374a05f354b1 currently {'job_id': '78592c3d-a1c9-4293-840f-374a05f354b1',
 'progress': 0,
 'started_at': datetime.datetime(2023, 5, 16, 16, 50, 14, 487057),
 'state': 'STARTED',
 'stopped_at': None,
 'submitted_at': datetime.datetime(2023, 5, 16, 16, 50, 14, 487000)}
job progress is now 25
job progress is now 50
job progress is now 75
job progress is now 90
job 78592c3d-a1c9-4293-840f-374a05f354b1 currently {'job_id': '78592c3d-a1c9-4293-840f-374a05f354b1',
 'progress': 100,
 'started_at': None,
 'state': 'SUCCESS',
 'stopped_at': datetime.datetime(2023, 5, 16, 16, 50, 54, 806866),
 'submitted_at': datetime.datetime(2023, 5, 16, 16, 50, 54, 806815)}


In [51]:
assert all(job.state == "SUCCESS" for job in job_statuses)


In [52]:
await get_job_result(solver, jobs[0])

{'job_id': '93c95f6c-8f35-4e11-bf5c-6e5d0e7d17a8',
 'results': {'out_1': {'checksum': 'd48be2403f4719ed3293c6569e45861c',
                       'content_type': 'application/octet-stream',
                       'filename': 'outputController.dat',
                       'id': 'f1ffd1a9-2a76-38a8-be2f-9e734d04643c'}}}

In [75]:
# jobs =await list_jobs(solver)
delete_jobs(jobs)
await asyncio.sleep(5)
jobs =await list_jobs(solver)
assert len(jobs) == 0, f"found {len(jobs)} jobs"

Deleting jobs:   0%|          | 0/1 [00:00<?, ?it/s]